In [1]:
# === 0) Setup: deps + repo (Kandinsky 2.2 training scripts live in diffusers/examples) ===
!pip -q install --upgrade pip
!pip -q install accelerate==0.33.0 transformers==4.44.2 datasets==2.21.0 safetensors==0.4.4 ftfy==6.2.0 xformers==0.0.27.post2
# install diffusers from source (recommended by HF guide for these scripts)
!git clone -q https://github.com/huggingface/diffusers
%cd diffusers
!pip -q install -e .
%cd examples/kandinsky2_2/text_to_image
!pip -q install -r requirements.txt

# Non-interactive Accelerate config (so it runs in notebooks)
from accelerate.utils import write_basic_config
write_basic_config()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 17.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
torchaudio 2.8.0+cu126 requires torch==2.8.0, but you have torch 2.4.0 which is incompatible.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.6.1 which is incompatible.
torchvision 0.23.0+cu126 requires torch==2.8.0, but you have torch 2.4.0 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which i

PosixPath('/root/.cache/huggingface/accelerate/default_config.yaml')

In [ ]:
# !rm -rf ARCH
# !rm -rf diffuser

In [2]:
# === 1) Unzip your dataset ===
# Expecting: /content/ARCH.zip with files image001.jpg + image001.txt ... image020.jpg + image020.txt
import zipfile, os, glob, textwrap, json
zip_path = "/content/ARCH.zip"
data_dir = "/content/ARCH"
os.makedirs(data_dir, exist_ok=True)
with zipfile.ZipFile(zip_path, 'r') as z:
    z.extractall(data_dir)

# Quick sanity check
print("Sample files:", sorted(os.listdir(data_dir))[:8])

# OPTIONAL: show how many matched pairs we have
jpgs = sorted(glob.glob(os.path.join(data_dir, "*.jpg")))
missing = [p for p in jpgs if not os.path.exists(p.replace(".jpg",".txt"))]
print(f"Found {len(jpgs)} JPGs, {len(missing)} missing captions (.txt). Missing list: {missing[:5]}")


Sample files: ['ARCH']
Found 0 JPGs, 0 missing captions (.txt). Missing list: []


In [3]:
# === 2) Small knobs for tiny datasets (20 samples) ===
# You can tweak these later if you want a bit more overfitting / sharpness.
PRIOR_STEPS    = 800      # prior learns text->image embeddings
DECODER_STEPS  = 1200     # decoder learns to render images
RESOLUTION     = 768      # Kandinsky 2.2 default; you can try 512 if low VRAM
BATCH_SIZE     = 1
GRAD_ACCUM     = 4        # effective batch size ~4
LR             = 1e-5
OUTPUT_PRIOR   = "/content/kandi22_prior_ARCH"
OUTPUT_DECODER = "/content/kandi22_decoder_ARCH"

# Pretrained model IDs (Hugging Face)
PRIOR_MODEL   = "kandinsky-community/kandinsky-2-2-prior"
DECODER_MODEL = "kandinsky-community/kandinsky-2-2-decoder"


In [ ]:
!pip install --upgrade pip
!pip install diffusers==0.18.2
!pip install transformers==4.30.2
!pip install accelerate==0.21.0
!pip install safetensors==0.3.1 datasets ftfy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 24.1 MB/s  0:00:00
  Attempting uninstall: diffusers
    Found existing installation: diffusers 0.36.0.dev0
    Uninstalling diffusers-0.36.0.dev0:
      Successfully uninstalled diffusers-0.36.0.dev0
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 87.0 MB/s  0:00:00
  error: subprocess-exited-with-error
  
  × Building wheel for tokenizers (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for tokenizers
Failed to build tokenizers
error: failed-wheel-build-for-install

× Failed to build installable wheels for some pyproject.toml based projects
╰─> tokenizers
  Attempting uninstall: accelerate
    Found existing installation: accelerate 

  Installing build dependencies ... canceledERROR: Operation cancelled by user
^C


In [8]:
# !pip install accelerate==0.21.0
# # !pip uninstall -y accelerate
# !pip install diffusers==0.18.2 transformers==4.30.2 safetensors==0.3.1

# !rm -f ~/.cache/huggingface/accelerate/default_config.yaml
# !accelerate config default
!pip install diffusers==0.18.2 transformers==4.30.2 accelerate==0.21.0 safetensors==0.3.1





In [1]:
# === 3) Train PRIOR (text -> image embeddings) ===
# Uses the official HF training script for Kandinsky 2.2 prior.
!accelerate launch --mixed_precision=fp16 train_text_to_image_prior.py \
  --train_data_dir "/content/ARCH" \
  --resolution {RESOLUTION} \
  --train_batch_size {BATCH_SIZE} \
  --gradient_accumulation_steps {GRAD_ACCUM} \
  --max_train_steps {PRIOR_STEPS} \
  --learning_rate {LR} \
  --max_grad_norm 1.0 \
  --lr_scheduler "constant" \
  --lr_warmup_steps 0 \
  --snr_gamma 5.0 \
  --pretrained_prior_model_name_or_path "{PRIOR_MODEL}" \
  --output_dir "{OUTPUT_PRIOR}"


2025-08-27 09:27:45.166430: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756286865.193034    5465 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756286865.201640    5465 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1756286865.224164    5465 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1756286865.224210    5465 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1756286865.224215    5465 computation_placer.cc:177] computation placer alr

In [2]:
from diffusers import KandinskyV22Pipeline, KandinskyV22PriorPipeline
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

# Load the prior pipeline (text → image embeddings)
prior = KandinskyV22PriorPipeline.from_pretrained(
    "kandinsky-community/kandinsky-2-2-prior",
    torch_dtype=torch.float16
).to(device)

# Load the main pipeline (image generation)
pipe = KandinskyV22Pipeline.from_pretrained(
    "kandinsky-community/kandinsky-2-2-decoder",
    torch_dtype=torch.float16
).to(device)

# # Example prompt
# prompt = "A futuristic cityscape at sunset, cinematic, ultra-detailed"

# # 1. Get image embeddings
# image_embeds = prior(prompt, guidance_scale=1.0).image_embeds

# # 2. Generate final image from embeddings
# image = pipe(image_embeds=image_embeds, guidance_scale=4.0, height=512, width=512).images[0]

# image.save("kandinsky_test.png")
# print("✅ Image saved as kandinsky_test.png")


model_index.json:   0%|          | 0.00/501 [00:00<?, ?B/s]

Fetching 13 files:   0%|          | 0/13 [00:00<?, ?it/s]

config.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

image_encoder/model.safetensors:   0%|          | 0.00/3.69G [00:00<?, ?B/s]

...

vocab.json: 0.00B [00:00, ?B/s]

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

Loading pipeline components...:   0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

TypeError: KandinskyV22Pipeline.__call__() missing 1 required positional argument: 'negative_image_embeds'

In [3]:
# from diffusers import KandinskyV22Pipeline, KandinskyV22PriorPipeline
# import torch

# device = "cuda" if torch.cuda.is_available() else "cpu"

# # Load prior (text → embeddings)
# prior = KandinskyV22PriorPipeline.from_pretrained(
#     "kandinsky-community/kandinsky-2-2-prior",
#     torch_dtype=torch.float16
# ).to(device)

# # Load decoder (embeddings → image)
# pipe = KandinskyV22Pipeline.from_pretrained(
#     "kandinsky-community/kandinsky-2-2-decoder",
#     torch_dtype=torch.float16
# ).to(device)

prompt = "A futuristic cityscape at sunset, cinematic, ultra-detailed"

# Get positive + negative embeddings
prior_output = prior(prompt, guidance_scale=1.0)
image_embeds = prior_output.image_embeds
negative_image_embeds = prior_output.negative_image_embeds

# Generate final image
image = pipe(
    image_embeds=image_embeds,
    negative_image_embeds=negative_image_embeds,
    guidance_scale=4.0,
    height=512,
    width=512
).images[0]

image.save("kandinsky_test.png")
print("✅ Image saved as kandinsky_test.png")


  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

✅ Image saved as kandinsky_test.png


In [1]:
# !rm -rf ARCH

In [ ]:
# --------------------------
# 1.5 Attach LoRA Attn Processors
# --------------------------
print("Attaching LoRA attention processors to UNet (if available)...")
from diffusers.models.attention_processor import LoRAAttnProcessor
attn_processors = {}
failed_attach = False
try:
    for k, proc in unet.attn_processors.items():
        # try get metadata
        h = getattr(proc, "hidden_size", None)
        cross = getattr(proc, "cross_attention_dim", None)
        try:
            l = LoRAAttnProcessor(hidden_size=h, cross_attention_dim=cross, rank=8, alpha=16)
            attn_processors[k] = l
        except Exception:
            attn_processors[k] = proc
    unet.set_attn_processor(attn_processors)
    print("LoRA processors set (where possible).")
except Exception as e:
    print("Could not attach LoRA processors automatically:", e)
    failed_attach = True

# --------------------------
# 1.6 Freeze everything except LoRA params
# --------------------------
n_trainable = 0
for name, p in unet.named_parameters():
    if "lora" in name.lower():
        p.requires_grad = True
        n_trainable += p.numel()
    else:
        p.requires_grad = False
print("Trainable LoRA params:", n_trainable)

if n_trainable == 0:
    raise SystemExit("No LoRA parameters detected to train. The LoRA attachment step failed. Paste the log if this happens.")

# --------------------------
# 1.7 Dataset -> latents (encode images using VAE)
# --------------------------
transform = T.Compose([T.Resize((RESOLUTION, RESOLUTION)), T.ToTensor()])

class SDLatentDataset(Dataset):
    def __init__(self, pairs, vae, transform, device):
        self.pairs = pairs
        self.vae = vae
        self.transform = transform
        self.device = device
    def __len__(self):
        return len(self.pairs)
    def __getitem__(self, idx):
        img_path, txt_path = self.pairs[idx]
        img = Image.open(img_path).convert("RGB")
        img = self.transform(img)
        caption = read_txt(txt_path)
        # encode to latents
        with torch.no_grad():
            inp = 2.0 * img.unsqueeze(0).to(self.device) - 1.0  # [-1,1]
            enc = self.vae.encode(inp.half())
            lat = enc.latent_dist.sample() * getattr(self.vae.config, "scaling_factor", 0.18215)
            lat = lat.squeeze(0).float()
        return {"latents": lat, "caption": caption}

dataset = SDLatentDataset(pairs, vae, transform, DEVICE)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

# --------------------------
# 1.8 Optimizer & scaler
# --------------------------
optimizer = torch.optim.AdamW([p for p in unet.parameters() if p.requires_grad], lr=LR)
scaler = torch.cuda.amp.GradScaler(enabled=(DEVICE=="cuda"))

# --------------------------
# 1.9 Training loop (denoising MSE on latents)
# --------------------------
print("Starting training loop...")
global_step = 0
unet.train()

while global_step < MAX_TRAIN_STEPS:
    for batch in dataloader:
        if global_step >= MAX_TRAIN_STEPS:
            break
        latents = batch["latents"].to(DEVICE)                 # [B, C, H, W]
        captions = [batch["caption"][i] for i in range(len(batch["caption"]))]  # list of strings

        # 1) get text encoder outputs (encoder_hidden_states)
        tokenized = tokenizer(captions, padding="max_length", truncation=True, max_length=77, return_tensors="pt").to(DEVICE)
        with torch.no_grad():
            encoder_hidden_states = text_encoder(tokenized.input_ids)[0].to(DEVICE)

        # 2) sample noise and timesteps
        noise = torch.randn_like(latents).to(DEVICE)
        timesteps = torch.randint(0, scheduler.num_train_timesteps, (latents.shape[0],), device=DEVICE).long()

        noisy_latents = scheduler.add_noise(latents, noise, timesteps)

        optimizer.zero_grad()
        with torch.cuda.amp.autocast(enabled=(DEVICE=="cuda")):
            pred = unet(noisy_latents.half(), timesteps, encoder_hidden_states=encoder_hidden_states.half()).sample
            loss = F.mse_loss(pred.float(), noise.float())

        if scaler.is_enabled():
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
        else:
            loss.backward()
            optimizer.step()

        if global_step % 50 == 0:
            print(f"step {global_step} loss: {loss.item():.4f}")

        global_step += 1

        # Save LoRA adapters periodically
        if global_step % SAVE_EVERY == 0 or global_step >= MAX_TRAIN_STEPS:
            save_dir = os.path.join(OUTPUT_DIR, f"lora_step_{global_step}")
            os.makedirs(save_dir, exist_ok=True)
            try:
                unet.save_attn_procs(save_dir)
                print("Saved LoRA adapters to", save_dir)
            except Exception as e:
                torch.save(unet.state_dict(), os.path.join(save_dir, "unet_state_dict.pt"))
                print("Saved fallback unet state_dict to", save_dir)
    # end epoch

print("Training finished. total steps:", global_step)
# final save
final_dir = os.path.join(OUTPUT_DIR, "final_lora")
os.makedirs(final_dir, exist_ok=True)
try:
    unet.save_attn_procs(final_dir)
    print("Saved final LoRA adapters to", final_dir)
except Exception as e:
    torch.save(unet.state_dict(), os.path.join(final_dir, "unet_state_dict.pt"))
    print("Saved final unet state_dict to", final_dir)

# --------------------------
# 1.10 Quick inference example
# --------------------------
print("\nRunning a quick sample generation using the LoRA adapters...")

from diffusers import StableDiffusionPipeline
pipe_sd = StableDiffusionPipeline.from_pretrained(MODEL_ID, torch_dtype=torch.float16).to(DEVICE)
# load adapters
try:
    pipe_sd.unet.load_attn_procs(final_dir)
    print("Loaded LoRA adapters into SD pipeline.")
except Exception as e:
    print("Could not load attn procs into SD pipeline:", e)

prompt = "A neat AWS architecture diagram with labeled boxes and arrows, minimal style"
out = pipe_sd(prompt, num_inference_steps=30, guidance_scale=7.5)
img = out.images[0]
out_path = os.path.join(OUTPUT_DIR, "sd_lora_sample.png")
img.save(out_path)
print("Saved sample:", out_path)

In [1]:
# ----------------------------
# Kandinsky Prior Fine-tuning (Student model) - single-cell
# ----------------------------
import os, glob, zipfile, json, random, time, shutil
from pathlib import Path
from PIL import Image
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
import numpy as np

# ----------------------------
# Config / hyperparams
# ----------------------------
# ZIP_PATH = "/content/ARCH.zip"   # upload this
DATA_DIR = "/content/ARCH/ARCH"       # will be created if zip exists
OUTPUT_DIR = "/content/kandi_prior_student"
os.makedirs(OUTPUT_DIR, exist_ok=True)

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
RESOLUTION = 512                # reduce to 256 if VRAM is tight
BATCH_SIZE = 1
NUM_EPOCHS = 50                 # small dataset; we'll do many epochs
LR = 2e-4
SEED = 42
torch.manual_seed(SEED)
random.seed(SEED)

print("Device:", DEVICE)

# ----------------------------
# Unzip dataset if needed
# ----------------------------
# if os.path.exists(ZIP_PATH) and (not os.path.exists(DATA_DIR) or len(os.listdir(DATA_DIR)) == 0):
#     print("Extracting", ZIP_PATH, "->", DATA_DIR)
#     with zipfile.ZipFile(ZIP_PATH, "r") as z:
#         z.extractall(DATA_DIR)
# else:
#     print("Using existing dataset dir:", DATA_DIR)

# ----------------------------
# Build list of (image_path, text)
# ----------------------------
pairs = []
for p in sorted(glob.glob(os.path.join(DATA_DIR, "*.jpg"))):
    t = p[:-4] + ".txt"
    if os.path.exists(t):
        with open(t, "r", encoding="utf-8") as f:
            caption = f.read().strip()
        pairs.append((p, caption))
print("Found pairs:", len(pairs))
if len(pairs) == 0:
    raise SystemExit("No image+txt pairs found. Ensure ARCH.zip contains imageNNN.jpg + imageNNN.txt")

# ----------------------------
# Load Kandinsky prior + decoder pipelines
# ----------------------------
from diffusers import KandinskyV22PriorPipeline, KandinskyV22Pipeline
print("Loading Kandinsky prior + decoder (this may take a minute)...")
prior = None
decoder = None
try:
    prior = KandinskyV22PriorPipeline.from_pretrained("kandinsky-community/kandinsky-2-2-prior", torch_dtype=torch.float16).to(DEVICE)
    decoder = KandinskyV22Pipeline.from_pretrained("kandinsky-community/kandinsky-2-2-decoder", torch_dtype=torch.float16).to(DEVICE)
    print("Loaded Kandinsky pipelines.")
except Exception as e:
    print("Warning: failed to load Kandinsky pipelines:", e)
    # we will try to continue with what we have

# ----------------------------
# Inspect decoder to find an image encoder (MOVQ or similar)
# ----------------------------
image_encoder = None
if decoder is not None:
    # print attributes to help debug if needed
    attrs = [k for k in decoder.__dict__.keys() if not k.startswith("_")]
    print("Decoder attributes:", attrs)
    # common candidates: 'movq', 'movq_encoder', 'movq_model', 'image_encoder'
    for cand in ["movq", "movq_model", "image_encoder", "movq_encoder"]:
        if hasattr(decoder, cand):
            image_encoder = getattr(decoder, cand)
            print("Found image encoder at decoder." + cand)
            break

# ----------------------------
# Fallback: try prior pipeline to produce embeddings from text, and decoder.movq to produce image embeddings
# ----------------------------
# We want to produce target embeddings for each image:
# strategy: if decoder has a movq/image_encoder -> use it to produce target embeddings from images.
# Otherwise we'll try other heuristics (fall back to prior outputs as targets - less ideal).
def encode_image_to_target(image_pil):
    """
    Return a torch.Tensor target embedding for the given PIL image.
    Tries multiple methods depending on what decoder exposes.
    """
    # 1) try movq / image_encoder if available
    if image_encoder is not None:
        # many MOVQ-like encoders accept numpy array or torch tensor; try safe calls
        try:
            # some MOVQ accept PIL image directly
            out = image_encoder(image_pil)
            # if out is a dict with 'image_embeds' or 'image_latents'
            if isinstance(out, dict):
                for key in ("image_embeds", "image_embeddings", "embeddings", "image_latent"):
                    if key in out:
                        emb = out[key]
                        return torch.as_tensor(emb).to(torch.float32).to(DEVICE)
                # else try first value
                first = next(iter(out.values()))
                return torch.as_tensor(first).to(torch.float32).to(DEVICE)
            # if returns torch tensor / numpy array
            if isinstance(out, torch.Tensor):
                return out.to(torch.float32).to(DEVICE)
            if isinstance(out, np.ndarray):
                return torch.from_numpy(out).float().to(DEVICE)
        except Exception as e:
            # try alternative call signatures: convert image->numpy then call
            try:
                arr = np.array(image_pil)
                out = image_encoder(arr)
                if isinstance(out, (torch.Tensor, np.ndarray)):
                    return torch.as_tensor(out).float().to(DEVICE)
            except Exception as e2:
                print("image_encoder calls failed:", e, "|", e2)
    # 2) try prior pipeline's encode route (not ideal)
    if prior is not None:
        try:
            # Some prior pipelines can accept image and produce embeddings (rare). Try safe call.
            prior_out = prior(image=image_pil)
            # pick plausible keys
            for key in ("image_embeds", "image_embeddings", "embeddings"):
                if key in prior_out:
                    emb = prior_out[key]
                    return torch.as_tensor(emb).to(torch.float32).to(DEVICE)
        except Exception:
            pass
    # 3) fallback: use decoder pipelines movq via decoding image -> run full pipeline and extract intermediate if possible
    raise RuntimeError("No supported image encoder found in decoder/prior. Cannot produce image targets automatically.")

# ----------------------------
# Build dataset of (tokenized_prompt, target_embedding)
# ----------------------------
# We need a tokenizer + text encoder to produce text features. Kandinsky prior pipeline typically does its own tokenization,
# but we will use a CLIP text encoder (stable, compatible) and train a small MLP that maps that text encoder's outputs -> target embeddings.
from transformers import CLIPTokenizer, CLIPTextModel

# Use CLIP tokenizer + text encoder from a stable model that matches Kandinsky's tokenizer style
# This should work well for diagram prompts.
CLIP_MODEL = "openai/clip-vit-large-patch14"  # common CLIP
print("Loading CLIP tokenizer + text encoder (this may take a moment)...")
tokenizer = CLIPTokenizer.from_pretrained(CLIP_MODEL)
text_encoder = CLIPTextModel.from_pretrained(CLIP_MODEL).to(DEVICE)
text_encoder.eval()

# Prepare transform for images
transform = T.Compose([T.Resize((RESOLUTION, RESOLUTION)), T.ToTensor()])

# Build target embeddings for each sample (this will raise if no image encoder)
examples = []
print("Encoding target image embeddings for each sample (this may take time)...")
for img_path, caption in pairs:
    pil = Image.open(img_path).convert("RGB")
    # try to extract target embedding
    try:
        target = encode_image_to_target(pil)               # torch tensor on DEVICE
    except RuntimeError as e:
        # no image encoder; fallback: use prior to produce image embeds from text (not ideal)
        if prior is not None:
            print("Falling back to using prior(prompt) as embedding target for caption:", caption)
            with torch.no_grad():
                p_out = prior(caption, guidance_scale=1.0)
                target = p_out.image_embeds
                if isinstance(target, torch.Tensor):
                    target = target.to(torch.float32).to(DEVICE)
                else:
                    target = torch.as_tensor(np.array(target)).to(torch.float32).to(DEVICE)
        else:
            raise SystemExit("No image encoder available and prior unavailable; cannot build training targets. Paste decoder attributes for debugging.")
    # store
    examples.append({"image_path": img_path, "caption": caption, "target_emb": target.cpu().numpy()})

print("Built", len(examples), "training examples.")

# ----------------------------
# Create a PyTorch Dataset that returns text features and target embeddings
# ----------------------------
class PriorStudentDataset(Dataset):
    def __init__(self, examples, tokenizer, text_encoder, device):
        self.examples = examples
        self.tokenizer = tokenizer
        self.text_encoder = text_encoder
        self.device = device
    def __len__(self):
        return len(self.examples)
    def __getitem__(self, idx):
        ex = self.examples[idx]
        caption = ex["caption"]
        # tokenize
        enc = self.tokenizer(caption, padding="max_length", truncation=True, max_length=77, return_tensors="pt")
        input_ids = enc.input_ids.squeeze(0)
        attention_mask = enc.attention_mask.squeeze(0)
        # compute text_encoder output (pooled or last_hidden_state)
        with torch.no_grad():
            out = self.text_encoder(input_ids.unsqueeze(0).to(self.device))[0]  # [1, seq, dim]
            # get pooled vector: mean pooling over tokens (or use [CLS] depending on model)
            txt_feat = out.mean(dim=1).squeeze(0).cpu().numpy()  # CPU numpy
        target = ex["target_emb"]  # numpy
        return {"text_feat": torch.from_numpy(txt_feat).float(), "target": torch.from_numpy(target).float()}

dataset = PriorStudentDataset(examples, tokenizer, text_encoder, DEVICE)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

# ----------------------------
# Build small student model: MLP mapping text_feat_dim -> target_emb_dim
# ----------------------------
# infer dims from first example
sample = dataset[0]
text_dim = sample["text_feat"].shape[0]
target_dim = sample["target"].shape[0]
print("Text feature dim:", text_dim, "Target embedding dim:", target_dim)

class StudentMLP(nn.Module):
    def __init__(self, in_dim, out_dim, hidden=2048):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(in_dim, hidden),
            nn.GELU(),
            nn.Linear(hidden, hidden),
            nn.GELU(),
            nn.Linear(hidden, out_dim)
        )
    def forward(self, x):
        return self.net(x)

student = StudentMLP(text_dim, target_dim).to(DEVICE)
optimizer = torch.optim.AdamW(student.parameters(), lr=LR)

# ----------------------------
# Training loop (MSE)
# ----------------------------
print("Training student model to map text features -> image embeddings...")
student.train()
for epoch in range(NUM_EPOCHS):
    total_loss = 0.0
    count = 0
    for batch in dataloader:
        txtf = batch["text_feat"].to(DEVICE)   # [B, text_dim]
        tgt = batch["target"].to(DEVICE)       # [B, target_dim]
        pred = student(txtf)
        loss = nn.functional.mse_loss(pred, tgt)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        count += 1
    avg = total_loss / max(1, count)
    if (epoch+1) % 5 == 0 or epoch==0:
        print(f"Epoch {epoch+1}/{NUM_EPOCHS} avg_loss={avg:.6f}")
print("Training finished.")

# ----------------------------
# Save the student model and metadata
# ----------------------------
torch.save(student.state_dict(), os.path.join(OUTPUT_DIR, "prior_student.pt"))
meta = {
    "text_dim": text_dim,
    "target_dim": target_dim,
    "tokenizer": tokenizer.name_or_path if hasattr(tokenizer, "name_or_path") else str(tokenizer),
    "clip_model": CLIP_MODEL,
    "device": DEVICE,
}
with open(os.path.join(OUTPUT_DIR, "prior_student_meta.json"), "w") as f:
    json.dump(meta, f)
print("Saved student model to", OUTPUT_DIR)

# ----------------------------
# Inference: how to use the student model with Kandinsky decoder
# ----------------------------
print("\nUSAGE EXAMPLE (run in a new cell):\n")
print("""
from diffusers import KandinskyV22PriorPipeline, KandinskyV22Pipeline
import torch, json
from transformers import CLIPTokenizer, CLIPTextModel

device = 'cuda' if torch.cuda.is_available() else 'cpu'
# load student's weights + tokenizer info
student = StudentMLP({text_dim}, {target_dim})
student.load_state_dict(torch.load('/content/kandi_prior_student/prior_student.pt', map_location=device))
student.to(device).eval()

tokenizer = CLIPTokenizer.from_pretrained('{CLIP_MODEL}')
text_encoder = CLIPTextModel.from_pretrained('{CLIP_MODEL}').to(device).eval()

prior = KandinskyV22PriorPipeline.from_pretrained('kandinsky-community/kandinsky-2-2-prior', torch_dtype=torch.float16).to(device)
decoder = KandinskyV22Pipeline.from_pretrained('kandinsky-community/kandinsky-2-2-decoder', torch_dtype=torch.float16).to(device)

# generate
prompt = "a neat AWS architecture diagram with boxes and arrows"
enc = tokenizer(prompt, padding='max_length', truncation=True, max_length=77, return_tensors='pt')
with torch.no_grad():
    txt_feat = text_encoder(enc.input_ids.to(device))[0].mean(dim=1)   # [1, text_dim]
    image_emb = student(txt_feat)   # [1, target_dim]
# depending on decoder's expected shape/scale you might need to convert to numpy or adjust dtype
# If prior_out expects batch dim, pass image_emb as image_embeds; create a dummy negative_image_embeds (zeros)
neg = torch.zeros_like(image_emb).to(device)
out = decoder(image_embeds=image_emb.half(), negative_image_embeds=neg.half(), guidance_scale=4.0, height={RESOLUTION}, width={RESOLUTION})
out.images[0].save('/content/kandi_student_output.png')
print('Saved /content/kandi_student_output.png')
""".format(text_dim=text_dim, target_dim=target_dim, CLIP_MODEL=CLIP_MODEL, RESOLUTION=RESOLUTION))

print("\nDone. Files saved to:", OUTPUT_DIR)


Device: cuda
Found pairs: 20
Loading Kandinsky prior + decoder (this may take a minute)...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

Loading pipeline components...:   0%|          | 0/3 [00:00<?, ?it/s]

Loaded Kandinsky pipelines.
Decoder attributes: ['unet', 'scheduler', 'movq', 'movq_scale_factor']
Found image encoder at decoder.movq
Loading CLIP tokenizer + text encoder (this may take a moment)...


...

image_encoder calls failed: conv2d() received an invalid combination of arguments - got (Image, Parameter, Parameter, tuple, tuple, tuple, int), but expected one of:
 * (Tensor input, Tensor weight, Tensor bias = None, tuple of ints stride = 1, tuple of ints padding = 0, tuple of ints dilation = 1, int groups = 1)
      didn't match because some of the arguments have invalid types: (!Image!, !Parameter!, !Parameter!, !tuple of (int, int)!, !tuple of (int, int)!, !tuple of (int, int)!, !int!)
 * (Tensor input, Tensor weight, Tensor bias = None, tuple of ints stride = 1, str padding = "valid", tuple of ints dilation = 1, int groups = 1)
      didn't match because some of the arguments have invalid types: (!Image!, !Parameter!, !Parameter!, !tuple of (int, int)!, !tuple of (int, int)!, !tuple of (int, int)!, !int!)
 | conv2d() received an invalid combination of arguments - got (numpy.ndarray, Parameter, Parameter, tuple, tuple, tuple, int), but expected one of:
 * (Tensor input, Tensor we

  0%|          | 0/25 [00:00<?, ?it/s]

Built 20 training examples.
Text feature dim: 768 Target embedding dim: 1
Training student model to map text features -> image embeddings...


/tmp/ipython-input-1797990229.py:251: UserWarning: Using a target size (torch.Size([1, 1, 1280])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = nn.functional.mse_loss(pred, tgt)


Epoch 1/50 avg_loss=1.358158
Epoch 5/50 avg_loss=1.269472
Epoch 10/50 avg_loss=1.258755
Epoch 15/50 avg_loss=1.258164
Epoch 20/50 avg_loss=1.258142
Epoch 25/50 avg_loss=1.258140
Epoch 30/50 avg_loss=1.258141
Epoch 35/50 avg_loss=1.258162
Epoch 40/50 avg_loss=1.258342
Epoch 45/50 avg_loss=1.260769
Epoch 50/50 avg_loss=1.262184
Training finished.
Saved student model to /content/kandi_prior_student

USAGE EXAMPLE (run in a new cell):


from diffusers import KandinskyV22PriorPipeline, KandinskyV22Pipeline
import torch, json
from transformers import CLIPTokenizer, CLIPTextModel

device = 'cuda' if torch.cuda.is_available() else 'cpu'
# load student's weights + tokenizer info
student = StudentMLP(768, 1)
student.load_state_dict(torch.load('/content/kandi_prior_student/prior_student.pt', map_location=device))
student.to(device).eval()

tokenizer = CLIPTokenizer.from_pretrained('openai/clip-vit-large-patch14')
text_encoder = CLIPTextModel.from_pretrained('openai/clip-vit-large-patch14').to(dev

In [5]:
pip install git+https://github.com/ai-forever/Kandinsky-2.git


  Cloning https://github.com/ai-forever/Kandinsky-2.git to /tmp/pip-req-build-9kxpjwlj
  Running command git clone --filter=blob:none --quiet https://github.com/ai-forever/Kandinsky-2.git /tmp/pip-req-build-9kxpjwlj
  Resolved https://github.com/ai-forever/Kandinsky-2.git to commit aeefc1ce3a989eefe7c99d6a02cce44318c4d210
  Preparing metadata (setup.py) ... done
  Using cached ftfy-6.3.1-py3-none-any.whl.metadata (7.3 kB)
Using cached ftfy-6.3.1-py3-none-any.whl (44 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 828.2/828.2 kB 24.9 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.0/983.0 kB 44.3 MB/s  0:00:00
  DEPRECATION: Building 'kandinsky2' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the s

In [2]:
import torch
import json

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# Load weights
model_path = "/content/kandi_prior_student/prior_student.pt"
meta_path = "/content/kandi_prior_student/prior_student_meta.json"

# Load checkpoint
checkpoint = torch.load(model_path, map_location=DEVICE)

# Inspect meta info
with open(meta_path, "r") as f:
    meta = json.load(f)

print("✅ Loaded fine-tuned prior")
print("Meta:", meta.keys())
print("Checkpoint keys:", checkpoint.keys())


✅ Loaded fine-tuned prior
Meta: dict_keys(['text_dim', 'target_dim', 'tokenizer', 'clip_model', 'device'])
Checkpoint keys: odict_keys(['net.0.weight', 'net.0.bias', 'net.2.weight', 'net.2.bias', 'net.4.weight', 'net.4.bias'])


In [4]:
import torch
from diffusers import KandinskyV22PriorPipeline, KandinskyV22Pipeline

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# --- 1. Load the official Kandinsky prior ---
base_prior = KandinskyV22PriorPipeline.from_pretrained(
    "kandinsky-community/kandinsky-2-2-prior",
    torch_dtype=torch.float16 if DEVICE=="cuda" else torch.float32
).to(DEVICE)

# --- 2. Load your fine-tuned weights ---
ckpt_path = "/content/kandi_prior_student/prior_student.pt"
state_dict = torch.load(ckpt_path, map_location=DEVICE)

# update prior UNet with your fine-tuned weights
missing, unexpected = base_prior.prior.load_state_dict(state_dict, strict=False)
print("✅ Fine-tuned prior loaded. Missing:", missing, "Unexpected:", unexpected)

# --- 3. Load Kandinsky decoder (official, not fine-tuned) ---
decoder = KandinskyV22Pipeline.from_pretrained(
    "kandinsky-community/kandinsky-2-2-decoder",
    torch_dtype=torch.float16 if DEVICE=="cuda" else torch.float32
).to(DEVICE)

# --- 4. Run text → image with your fine-tuned prior ---
prompt = "An architectural diagram of a cloud data pipeline with AWS icons"

# get embedding from fine-tuned prior
image_embeds = base_prior(prompt).image_embeds

# decode embedding into image
images = decoder(
    image_embeds=image_embeds,
    num_inference_steps=50,
    guidance_scale=7.5
).images

# save result
images[0].save("finetuned_test.png")
print("✅ Image saved as finetuned_test.png")


Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

✅ Fine-tuned prior loaded. Missing: ['positional_embedding', 'prd_embedding', 'clip_mean', 'clip_std', 'time_embedding.linear_1.weight', 'time_embedding.linear_1.bias', 'time_embedding.linear_2.weight', 'time_embedding.linear_2.bias', 'proj_in.weight', 'proj_in.bias', 'embedding_proj.weight', 'embedding_proj.bias', 'encoder_hidden_states_proj.weight', 'encoder_hidden_states_proj.bias', 'transformer_blocks.0.norm1.weight', 'transformer_blocks.0.norm1.bias', 'transformer_blocks.0.attn1.to_q.weight', 'transformer_blocks.0.attn1.to_q.bias', 'transformer_blocks.0.attn1.to_k.weight', 'transformer_blocks.0.attn1.to_k.bias', 'transformer_blocks.0.attn1.to_v.weight', 'transformer_blocks.0.attn1.to_v.bias', 'transformer_blocks.0.attn1.to_out.0.weight', 'transformer_blocks.0.attn1.to_out.0.bias', 'transformer_blocks.0.norm3.weight', 'transformer_blocks.0.norm3.bias', 'transformer_blocks.0.ff.net.0.proj.weight', 'transformer_blocks.0.ff.net.0.proj.bias', 'transformer_blocks.0.ff.net.2.weight', 'tr

Loading pipeline components...:   0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

TypeError: KandinskyV22Pipeline.__call__() missing 1 required positional argument: 'negative_image_embeds'

In [6]:
# decoder = KandinskyV22Pipeline.from_pretrained(
#     "kandinsky-community/kandinsky-2-2-decoder",
#     torch_dtype=torch.float16 if DEVICE=="cuda" else torch.float32
# ).to(DEVICE)

# --- 4. Run text → image with your fine-tuned prior ---
prompt = "An architectural diagram of a cloud data pipeline with AWS icons"
negative_prompt = ""  # you can set "bad quality, blurry" etc. if needed

# get positive & negative embeddings from fine-tuned prior
prior_out = base_prior(prompt)
neg_prior_out = base_prior(negative_prompt)

# decode embeddings into image
images = decoder(
    image_embeds=prior_out.image_embeds,
    negative_image_embeds=neg_prior_out.image_embeds,
    num_inference_steps=50,
    guidance_scale=7.5
).images

# save result
images[0].save("finetuned_test.png")
print("✅ Image saved as finetuned_test.png")

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

✅ Image saved as finetuned_test.png


In [7]:
# decoder = KandinskyV22Pipeline.from_pretrained(
#     "kandinsky-community/kandinsky-2-2-decoder",
#     torch_dtype=torch.float16 if DEVICE=="cuda" else torch.float32
# ).to(DEVICE)

# --- 4. Run text → image with your fine-tuned prior ---
prompt = "An architectural diagram of a user data going to aws cloud and then to s3 bucket make it neat and clear"
negative_prompt = ""  # you can set "bad quality, blurry" etc. if needed

# get positive & negative embeddings from fine-tuned prior
prior_out = base_prior(prompt)
neg_prior_out = base_prior(negative_prompt)

# decode embeddings into image
images = decoder(
    image_embeds=prior_out.image_embeds,
    negative_image_embeds=neg_prior_out.image_embeds,
    num_inference_steps=50,
    guidance_scale=7.5
).images

# save result
images[0].save("finetuned_test.png")
print("✅ Image saved as finetuned_test.png")

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

✅ Image saved as finetuned_test.png
